In [1]:
# NOTES
# TODO: Setup environment variables in anaconda or use Python without anaconda to access exported evnironment vars 

# Setup

## Import Packages

In [2]:
# import packages
import os
import dotenv
from collections import Counter
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, TimestampType
from pyspark.sql.functions import col, year, month, weekofyear, unix_timestamp, udf
from hdfs import InsecureClient

## Load Environemnt Variables

In [3]:
# load environment variables
dotenv.load_dotenv()

SPARK_CUDF_JAR = os.getenv('SPARK_CUDF_JAR')
SPARK_RAPIDS_PLUGIN_JAR = os.getenv('SPARK_RAPIDS_PLUGIN_JAR')
namenode_URI = os.getenv('namenode_URI')
hadoop_user = os.getenv('hadoop_user')
hdfs_path = os.getenv('hdfs_path')

## Configure Spark, Connection to HDFS

In [4]:
# configure environment & instantiate spark
# one cpu available, total 8 cores, 2 cores per executor
# one gpu available, 4 concurent tasks for gpu (1/4), 90% of gpu memory allowed for pooling by RAPIDS
# possibly implement SparkConf.set method to set config('spark.executor.resource.gpu.amount', '0.25')
spark = SparkSession.builder \
    .master('local') \
    .appName('nyc_taxi') \
    .config('spark.jars', f'{SPARK_CUDF_JAR},{SPARK_RAPIDS_PLUGIN_JAR}') \
    .config('spark.plugins', 'com.nvidia.spark.SQLPlugin') \
    .config('spark.rapids.sql.enabled', 'true') \
    .config('spark.rapids.sql.incompatibleOps.enabled', 'true') \
    .config('spark.executor.cores', '2') \
    .config('spark.task.cpus', '1') \
    .config('spark.rapids.memory.gpu.pooling.enabled', 'true') \
    .config('spark.rapids.memory.gpu.allocFraction', '0.9') \
    .getOrCreate()


2021-10-21 18:25:25,427 WARN util.Utils: Your hostname, david-Z97X-Gaming-7 resolves to a loopback address: 127.0.1.1; using 192.168.0.159 instead (on interface enp42s0)
2021-10-21 18:25:25,427 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2021-10-21 18:25:25,599 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-10-21 18:25:29,529 ERROR rapids.RapidsExecutorPlugin: Exception in the executor plugin
ai.rapids.cudf.CudaException: unknown error
	at ai.rapids.cudf.Cuda.getDeviceCount(Native Method)
	at com.nvidia.spark.rapids.GpuDeviceManager$.findGpuAndAcquire(GpuDeviceManager.scala:78)
	at com.nvidia.spark.rapids.GpuDeviceManager$.$anonfun$initializeMemory$1(GpuDeviceManager.scala:322)
	at scala.runtime.java8.JFunction0$mcI$sp.apply(JFunction0$mcI$s

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

In [ ]:
# list files in hdfs directory, count files in directory
hdfs = InsecureClient(namenode_URI, user=hadoop_user)
fs = hdfs.list(hdfs_path)
print(fs)
print('number of files in dfs directory:', len(fs))

['yellow_taxi_2017-01.parquet', 'yellow_taxi_2017-02.parquet', 'yellow_taxi_2017-03.parquet', 'yellow_taxi_2017-04.parquet', 'yellow_taxi_2017-05.parquet', 'yellow_taxi_2017-06.parquet', 'yellow_taxi_2017-07.parquet', 'yellow_taxi_2017-08.parquet', 'yellow_taxi_2017-09.parquet', 'yellow_taxi_2017-10.parquet', 'yellow_taxi_2017-11.parquet', 'yellow_taxi_2017-12.parquet', 'yellow_taxi_2018-01.parquet', 'yellow_taxi_2018-02.parquet', 'yellow_taxi_2018-03.parquet', 'yellow_taxi_2018-04.parquet', 'yellow_taxi_2018-05.parquet', 'yellow_taxi_2018-06.parquet', 'yellow_taxi_2018-07.parquet', 'yellow_taxi_2018-08.parquet', 'yellow_taxi_2018-09.parquet', 'yellow_taxi_2018-10.parquet', 'yellow_taxi_2018-11.parquet', 'yellow_taxi_2018-12.parquet', 'yellow_taxi_2019-01.parquet', 'yellow_taxi_2019-02.parquet', 'yellow_taxi_2019-03.parquet', 'yellow_taxi_2019-04.parquet', 'yellow_taxi_2019-05.parquet', 'yellow_taxi_2019-06.parquet', 'yellow_taxi_2019-07.parquet', 'yellow_taxi_2019-08.parquet', 'yellow

# Data Ingestion

In [ ]:
# Get all column names in data
columns = []
for prqt in hdfs.list(hdfs_path):
    cols = spark.read \
        .options(header='true', inferschema='true') \
        .parquet(f'hdfs://localhost:9000/user/hadoop/input/{prqt}') \
        .columns
    columns.extend(cols)

# see what columns are similar between
counter = Counter(columns)
print(counter)

# lets take all columns that are present within all datasets (48 maximum columns count)
columns = [i[0] for i in counter.items() if i[1] == max([i[1]for i in counter.items()])]


Counter({'VendorID': 48, 'tpep_pickup_datetime': 48, 'tpep_dropoff_datetime': 48, 'passenger_count': 48, 'trip_distance': 48, 'RatecodeID': 48, 'store_and_fwd_flag': 48, 'PULocationID': 48, 'DOLocationID': 48, 'payment_type': 48, 'fare_amount': 48, 'extra': 48, 'mta_tax': 48, 'tip_amount': 48, 'tolls_amount': 48, 'improvement_surcharge': 48, 'total_amount': 48, 'congestion_surcharge': 24})


In [ ]:
# set the schema for data
customSchema = StructType([
    StructField('VendorID', LongType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', LongType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('RatecodeID', LongType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('PULocationID', LongType(), True),
    StructField('DOLocationID', LongType(), True),
    StructField('payment_type', LongType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True)
])


In [ ]:
# read in data
df = spark.read \
    .format('parquet') \
    .schema(customSchema) \
    .load('hdfs://localhost:9000/user/hadoop/input/*')


In [ ]:
# check final schema
df.printSchema()


root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [ ]:
# preview of concatenated dataframe size
print('number of rows: ', round(df.count() / 1e6, 2), 'Million',
      '\nnumber of columns: ', len(df.columns),
      '\ntotal count of data: ', round(df.count() * len(df.columns) / 1e9, 2), 'Billion')


number of rows:  325.35 Million 
number of columns:  17 
total count of data:  5.53 Billion


# Data Preparation

In [ ]:
# we should only have data on years 2017 - 2020
# df.select(year(df.tpep_pickup_datetime)).distinct().collect()[0][0]

# check how many rows of data are not within 2017-2020
years = ['2017', '2018', '2019', '2020']
# df.filter(~year(df.tpep_pickup_datetime).isin(years)).count()

# 113.5k relatively small amount of taxi rides compared to 325.3m, we can drop these rows
df = df.filter(year(df.tpep_pickup_datetime).isin(years))

In [ ]:
# check the months within the data
# df.select(month(df.tpep_pickup_datetime)).distinct().collect()[0][0]
# time 1m 30.6s

12

In [ ]:
# check if the years are correct like the pickup datetime
# df.select(year(df.tpep_dropoff_datetime)).distinct().collect()[0][0]

# there are extra rows with improper years, check the amount of bad data
# df.filter(~year(df.tpep_dropoff_datetime).isin(years)).count()

# only 75 rows, we can drop them
df = df.filter(year(df.tpep_dropoff_datetime).isin(years))

In [ ]:
# check the months within the dates
# df.select(month(df.tpep_dropoff_datetime)).distinct().collect()[0][0]
# time 1n 32s

12

In [ ]:
# user defined function for finding difference between two timestamps in minutes"
time_difference_udf = udf(lambda pickup, dropoff: (unix_timestamp(dropoff) - unix_timestamp(pickup)) / 60)

In [ ]:
# get attributes of dates
df = df.withColumn('pickup_year', year('tpep_pickup_datetime')) \
       .withColumn('pickup_month', month('tpep_pickup_datetime')) \
       .withColumn('pickup_week', weekofyear('tpep_pickup_datetime')) \
       .withColumn('ride_duration', time_difference_udf(col('tpep_pickup_datetime'), col('tpep_dropoff_datetime')))

In [71]:
df.take(10)

21/10/21 17:52:58 ERROR Executor: Exception in task 0.0 in stage 103.0 (TID 4522)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/s

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/david/anaconda3/envs/Spark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 200,

In [53]:
# number of rides per month
rides_per_month = df.select(
                        year("tpep_pickup_datetime").alias('year'),
                        month("tpep_pickup_datetime").alias('month'),
                        'fare_amount') \
                    .groupBy('year', 'month') \
                    .count() \
                    .toPandas()
# time: 33.5s
rides_per_month.head()

,year,month,count
0,2019,10,7242123
1,2017,3,10309482
2,2017,8,8433694
3,2020,6,550348
4,2019,5,7584383


In [54]:
# sum of fares by month-year
fare_by_year_month = df.select(
                            year("tpep_pickup_datetime").alias('year'),
                            month("tpep_pickup_datetime").alias('month'),
                            'fare_amount') \
                        .groupBy('year', 'month') \
                        .sum('fare_amount') \
                        .toPandas()
# time: 29.8s
fare_by_year_month.head()

,year,month,sum(fare_amount)
0,2019,10,9.870654e+07
1,2017,3,1.331977e+08
2,2017,8,1.110681e+08
3,2020,6,7.499550e+06
4,2019,5,1.019473e+08


In [56]:
# get total fares by pick up and drop off location pairs
fares_by_locations = df.groupBy('PULocationID', 'DOLocationID').sum('fare_amount').toPandas()
# time: 20.4s
fares_by_locations.head()

,PULocationID,DOLocationID,sum(fare_amount)
0,137,37,112307.40
1,142,144,480548.47
2,90,231,1527774.76
3,255,262,18660.00
4,234,144,1658256.19


# APPENDIX

```python
# RDD implementation of fares_by_locations (GPU not utilized)
fares_by_locations = df.select(['PULocationID', 'DOLocationID', 'fare_amount']).rdd \
                         .map(lambda x: ((x[0], x[1]), x[2])) \
                         .reduceByKey(lambda x, y: x + y) \
                         .toDF().toPandas()
time: 10m 53s
```